# Quantum Grid Search

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import Diagonal, MCMT, ZGate, XGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import Sampler
from qiskit_aer import Aer

### Quantum Binary Formulation

In [2]:
def f(x, A, b, c):
    """
    returns the values of the quadratic function x.T @ A @ x + b.T@x +c
    -------------------------------------------------------------------------------
    Parameters
        x : argument of the quadratic function
        A : matrix constant of the quadratic function x.T @ A @ x + b.T@x +c 
        b : vector constant of the quadratic function x.T @ A @ x + b.T@x +c
        c : scalar constant of the quadratic function
    Returns
        f(x) : float value of the quadratic function f(x) 
    """
    return x.T @ A @ x + b.T@x +c

def createCoefficientsF(A, b, p, n, removeDiag = True):
    """
    returns the coefficients A', b' of the quadratic function q.T @ A' @ q + b'.T@q +c.
    -------------------------------------------------------------------------------
    Parameters
        A         : matrix constant of the quadratic function x.T @ A @ x + b.T@x +c 
        b         : vector constant of the quadratic function x.T @ A @ x + b.T@x +c
        p         : number of cell in the grid for each dimensions
        n         : number of qubits used in the state representation
        removeDiag: boolean set to True if we want the diagonal of newA to be removed and added to newb
    Returns
        newA : matrix of the quadr. coeff. of the new problem.
        newb : vector of the linear coeff. of the new problem.
    """
    d = b.size
    
    newA = np.zeros((n*d, n*d))
    newb = np.zeros(n*d)

    for r in range(d):
        for l in range(d):
            for k in range(n):
                for u in range(n):
                    i = r*n+k
                    j = l*n+u

                    if i==j and removeDiag:
                        newb[i] = 2**(2*(n-1)-k-u)*A[r,l]/p**2
                    else:
                        newA[i,j] = 2**(2*(n-1)-k-u)*A[r,l]/p**2

    for j in range(d):
        for k in range(n):
            i = j*n+k

            newb[i] += 2**(n-1-k)*b[j]/p

    return newA, newb
    
def F(q, A, b, c, p, n):
    """
    returns the values of the quadratic function q.T @ A' @ q + b'.T@q +c. A' and b' are calculated 
    from A and b of the quadr. func. x.T @ A @ x + b.T@x +c
    -------------------------------------------------------------------------------
    Parameters
        q : binary arguments of the quadratic function
        A : matrix constant of the quadratic function x.T @ A @ x + b.T@x +c 
        b : vector constant of the quadratic function x.T @ A @ x + b.T@x +c
        c : scalar constant of the quadratic function
        p : number of cell in the grid for each dimensions
        n : number of qubits used in the state representation
    Returns
        F(q) : float value of the quadratic function F(q) = f(x) 
    """
    d = b.size
    
    newA, newb = createCoefficientsF(A, b, p, n, removeDiag = True)
    
    return q.T@newA@q + newb.T@q + c

In [3]:
def binary2int(binary_str):
    if binary_str =="":
        return 0
    return int(binary_str, 2)

def int2binary(int_num, n_bits):
    return bin(int_num)[2:].zfill(n_bits)

def float2int(number, p): #from float \in [0,1] to int corresponding to grid formulation
    return int(np.round(number*p))

def roundToClosestCorrespondingBin(num, smallestCorrespondingBin):
    return np.round(num / smallestCorrespondingBin) * smallestCorrespondingBin

def decimalPart2binary(decimalPart, nBitsDecPart):
    """return the decimal part of a number in binary"""
    den = np.sum(2**np.arange(0,nBitsDecPart,1))+1
    correspondingInt = int(np.round(decimalPart*den))
    return int2binary(correspondingInt, nBitsDecPart)

def binary2decimalPart(BinaryDecimalPart):
    """ return the corresponding binary part of a number from a binary string"""
    decimalPart  = binary2int(BinaryDecimalPart)
    decimalPart /= 1 + binary2int("1"*len(BinaryDecimalPart))
    return decimalPart

def binaryComma2float(binComma, nBitsIntegerPart):
    intPart = binary2int(binComma[:nBitsIntegerPart])
    decPart = binary2decimalPart(binComma[nBitsIntegerPart:])
    return intPart+decPart

def float2binaryComma(number, nBitsIntegerPart, nBitsDecPart):
    approxNumber = np.round(number / 2**(-nBitsDecPart)) * 2**(-nBitsDecPart)
    decPart, intPart = np.modf(approxNumber)
    return int2binary(int(intPart), nBitsIntegerPart) + decimalPart2binary(decPart, nBitsDecPart)

def binaryComma2real(binComma, nBitsIntegerPart): # signBit|IntPart|DecPart
    negPart = -2**nBitsIntegerPart*int(binComma[0])
    intPart = binary2int(binComma[1:nBitsIntegerPart+1])
    decPart = binary2decimalPart(binComma[nBitsIntegerPart+1:])
    return negPart+intPart+decPart

def real2binaryComma(number, nBitsIntegerPart, nBitsDecPart): # signBit|IntPart|DecPart
    if number<0:
        signBit = "1"
        number += 2**nBitsIntegerPart
    else:
        signBit = "0"
    approxNumber = np.round(number / 2**(-nBitsDecPart)) * 2**(-nBitsDecPart)
    decPart, intPart = np.modf(approxNumber)
    return signBit + int2binary(int(intPart), nBitsIntegerPart) + decimalPart2binary(decPart, nBitsDecPart)

def processGroverOutput(output, mQubits, nBitsIntegerPart):
    q = output[mQubits:][::-1]
    Fq = output[:mQubits]
    
    return q, binaryComma2real(Fq, nBitsIntegerPart)

### State Superposition

In [4]:
def UG(mBitsIntPart, mQubits, nc, a):
    """
    returns the control phase gate corresponding to a monomial in pol(x)
    -------------------------------------------------------------------------------
    Parameters
        mBitsIntPart : number of qubits of the integer part of the function values
        mQubits      : number of qubits used in the state representation of the function values
        nc           : number of control qubits
        a            : coefficient of the monomial
    Returns
        UG : Gate corresponding to phase gate
    """
    abs_a = abs(a)
    sign_a = np.sign(a)
    
    UGcircuit = QuantumCircuit(mQubits, name = f"$U_G({np.round(a,2)})$")

    totalBin_a = float2binaryComma(abs_a, mBitsIntPart, mQubits - mBitsIntPart-1)
    transformedInt_a = binary2int(totalBin_a)
    
    for i in range(mQubits):

        omega = sign_a*transformedInt_a*2**(1-mQubits)
        UGcircuit.p(2**(mQubits-i-1)*np.pi*omega, i)

    if nc>0:
        UG = UGcircuit.to_gate().control(nc)
    else:
        UG = UGcircuit.to_gate()

    return UG

def inv_QFT(mBitsIntPart, mQubits):
    """
    returns the gate that performs the inverse QFT on a state superposition of a mQubits quantum register.
    ------------------------------------------------------------------------------------------------------
    Parameters
        mBitsIntPart : number of qubits of the integer part of the function values
        mQubits      : number of qubits used in the state representation of the function values
    Returns
        invQFTgate   : inverse QFT gate
    """
    invQFTcircuit = QuantumCircuit(mQubits, name = "QFT*")
    for i in range(mQubits):
        invQFTcircuit.h(i)
        for j in range(i+1, mQubits):
            invQFTcircuit.cp(-np.pi * 2**(i-j), i, j)

    invQFTgate = invQFTcircuit.to_gate()
    return invQFTgate

In [5]:
def statesPhasePreparator(d, nQubits, mBitsIntPart, mQubits, A, b, c):
    """
    Returns the quantum circuit that initialize the arguments qubits 'q', the additional qubit 'anc'
    need for Grover's algorithm and stores the quadratic function values to the phase of the mQubits register.
    ------------------------------------------------------------------------------------------------------
    Parameters
        d            : dimension of the quadratic function F(q)
        nQubits      : number of qubits needed for the argument register.
        mBitsIntPart : number of qubits of the integer part of the function values
        mQubits      : number of qubits used in the state representation of the function values
        A            : quadratic coefficient matrix of the quantum formulation
        b            : vector coefficient matrix of the quantum formulation
        c            : constant matrix of the quantum formulation
    Returns
        inputCircuit : Phase initializatio circuit
    """
    inputArgs = QuantumRegister(d*nQubits, name = 'q')
    storageFuncVals = QuantumRegister(mQubits, name = 'F')
    ancillaQubit = QuantumRegister(1, name = "an")

    inputCircuit = QuantumCircuit(inputArgs, storageFuncVals,ancillaQubit)

    inputCircuit.h(inputArgs)
    inputCircuit.h(ancillaQubit)
    
    # QFT:
    inputCircuit.h(storageFuncVals)
    
    UGgate = UG(mBitsIntPart, mQubits, 0, c) #for the constant c
    inputCircuit.append(UGgate, [d*nQubits+ k for k in range(mQubits)])
    
    for i in range(nQubits*d): # for the matrix A
        for j in range(nQubits*d):
            if A[i,j] !=0:
                UGgate = UG(mBitsIntPart, mQubits, 2, A[i,j])
                inputCircuit.append(UGgate, [i, j] + [d*nQubits + k for k in range(mQubits)])
    
    for i in range(nQubits*d): # for the vector b
        if b[i] !=0:
            UGgate = UG(mBitsIntPart, mQubits, 1, b[i])
            inputCircuit.append(UGgate, [i] + [d*nQubits + k for k in range(mQubits)])

    return inputCircuit

def statesPreparator(d, nQubits, mBitsIntPart, mQubits, SPP):
    """
    Returns the quantum circuit that transform the phase initialization circuit to the superpostion of the 
    quadratic function values.
    This circuit is represente by A in the Grover'algorithm
    ------------------------------------------------------------------------------------------------------
    Parameters
        d            : dimension of the quadratic function F(q)
        nQubits      : number of qubits needed for the argument register.
        mBitsIntPart : number of qubits of the integer part of the function values
        mQubits      : number of qubits used in the state representation of the function values
        SPP          : quantum circuit of the phase initialization circuit
    Returns
        inputCircuit : Phase initializatio circuit
    """
    # Circuit creation:
    inputArgs = QuantumRegister(d*nQubits, name = 'q')
    storageFuncVals = QuantumRegister(mQubits, name = 'F')
    ancillaQubit = QuantumRegister(1, name = "an")

    inputCircuit = QuantumCircuit(inputArgs, storageFuncVals,ancillaQubit)

    inputCircuit.append(SPP.to_gate(label = "$SPP$"), range(d*nQubits + mQubits+1))

    # Inverse QFT: 
    inv_QFT_gate = inv_QFT(mBitsIntPart, mQubits)
    inputCircuit.append(inv_QFT_gate, [d*nQubits + k for k in range(mQubits)])
    
    return inputCircuit

def diffusor(d, nQubits, mQubits):
    """
    Returns the quantum circuit of the diffusor of the Grover's algorithm
    ------------------------------------------------------------------------------------------------------
    Parameters
        d      : dimension of the quadratic function F(q)
        nQubits: number of qubits needed for the argument register.
        mQubits: number of qubits used in the state representation of the function values
    Returns
        D      : Grover's diffusor
    """
    
    D = QuantumCircuit(d*nQubits+mQubits +1)

    D.x(range(d*nQubits+mQubits +1))
    D.h(-1)
    D.mcx([i for i in range(d*nQubits+mQubits)], -1)
    D.h(-1)
    D.x(range(d*nQubits+mQubits +1))
    
    return D

def oracle(mQubits):
    """
    Returns the quantum circuit of the Grover's oracle
    ------------------------------------------------------------------------------------------------------
    Parameters
        mQubits: number of qubits used in the state representation of the function values
    Returns
        O      : Grover's oracle
    """

    O = QuantumCircuit(mQubits + 1)

    O.cz(mQubits-1, mQubits)

    return O

### Grover's algorithm

In [6]:
def GroverIterator(d, nQubits, mQubits, SP, inv_SP, O, D, iteGrover):
    """
    Returns the quantum circuit of the Grover's iteration (OA*DA)^k
    ------------------------------------------------------------------------------------------------------
    Parameters
        d        : dimension of the quadratic function F(q)
        nQubits  : number of qubits needed for the argument register.
        mQubits  : number of qubits used in the state representation of the function values
        SP       : State preparation quantum circuit
        inv_SP   : inverse of the state preparation circuit
        O        : Grover's oracle
        D        : Grover's diffusor
        iteGrover: number of Grover iteration
    Returns
        G        : Grover's iteration circuit
    """

    G = QuantumCircuit(d*nQubits + mQubits + 1, name = "$G$")

    for i in range(max(1, iteGrover)):
        G.append(O.to_gate(label = "$O$"), range(d*nQubits, d*nQubits + mQubits+1))
        G.append(inv_SP.to_gate(label = "$A*$"), range(d*nQubits + mQubits+1))
        G.append(D.to_gate(label = "$D$"), range(d*nQubits + mQubits+1))
        G.append(SP.to_gate(label = "$A$"), range(d*nQubits + mQubits+1))

    return G

def GroverAlgorithm(offset, SPP, O, D, iteGrover, d, nQubits, mQubits, mBitsIntPart, passManager, sampler):
    """
    Executes Grover's Algorithm to find, with the highest probability, a state representing a value smaller than the current smallest state.

    Parameters:
        offset (int): Offset used in the UG gate.
        SPP (QuantumCircuit): Circuit that creates the superposition of feasible states for the problem.
        O (QuantumCircuit): Oracle circuit used in Grover's Algorithm.
        D (QuantumCircuit): Diffusion operator circuit used in Grover's Algorithm.
        iteGrover (int): Number of Grover iterations to perform.
        d (int): Dimension of the problem.
        nQubits (int): Number of qubits used for the binary arguments.
        mQubits (int): Number of qubits used for the integer function value.
        mBitsIntPart (int): Number of bits for the integer part of the function value.
        passManager (PassManager): Qiskit pass manager for circuit optimization.
        sampler (Sampler): Qiskit sampler for running the circuit.

    Returns:
        str: Binary string representing the state with the global maximum value.
    """
    
    UGgate = UG(mBitsIntPart, mQubits, 0, -1*offset)
    SPP.append(UGgate, [d*nQubits+ k for k in range(mQubits)])

    SP = statesPreparator(d, nQubits, mBitsIntPart, mQubits, SPP)
    inv_SP = SP.inverse()

    GroverIterations = GroverIterator(d, nQubits, mQubits, SP, inv_SP, O, D, iteGrover)
    
    GCQReg = QuantumRegister(d*nQubits+mQubits+1)
    GCCReg = ClassicalRegister(d*nQubits+mQubits)
 
    GCircuit = QuantumCircuit(GCQReg, GCCReg)

    GCircuit.append(SP.to_gate(label = "$A$"), range(d*nQubits + mQubits+1))
    
    GCircuit.append(GroverIterations.to_gate(label = f"$G$^{iteGrover}"), range(d*nQubits + mQubits+1))

    GCircuit.measure(GCQReg[:d*nQubits + mQubits], GCCReg)

    job = pm.run(GCircuit)
    result = sampler.run(job, skip_transpilation=True, shots = 1).result()

    return int2binary(list(result.quasi_dists[0].keys())[0], d*nQubits+mQubits)

    

### Grover Adaptive Search

In [7]:
def GroverAdaptiveSearch(SPP, d, nQubits, mQubits, mBitIntPart, nElements, initialX, iteGroverMax, passManager, sampler):
    """
    Executes an adaptive version of Grover's Algorithm to find the global minimum state of a state superposition.

    Parameters:
        SPP (QuantumCircuit): Circuit that creates the superposition of feasible states for the problem.
        d (int): Dimension of the problem.
        nQubits (int): Number of qubits used for the binary arguments.
        mQubits (int): Number of qubits used for the integer function value.
        mBitsIntPart (int): Number of bits for the integer part of the function value.
        nElements (int): Number of elements in the state superposition.
        initialX (tuple): Initial value (q_min, offset) considered as the current maximum to start the search.
        iteGroverMax (int): Maximum number of Grover subroutine iterations.
        passManager (PassManager): Qiskit pass manager for circuit optimization.
        sampler (Sampler): Qiskit sampler for running the circuit.

    Returns:
        tuple: A tuple (q_min, tot_offset) where q_min is the state representing the global minimum and tot_offset is the offset value.
    """
    
    iteMax = int(np.floor(np.pi / (4 * np.arcsin(1/np.sqrt(nElements)))))
    iteGroverChoice = np.arange(1, iteMax+1)

    q_min, offset = initialX
    tot_offset = offset

    O = oracle(mQubits)
    D = diffusor(d, nQubits, mQubits)
    
    Found = True

    count = 0
    while Found:
        Found = False
        T = 1
        while T <= iteGroverMax:
            iteGrover = int(np.random.choice(iteGroverChoice[0:min(T,iteMax)])) #see https://www.youtube.com/watch?v=hnpjC8WQVrQ (50')

            SPPcopy =  SPP.copy()
            state_i = GroverAlgorithm(tot_offset, SPPcopy, O, D, iteGrover, d, nQubits, mQubits, mBitsIntPart, passManager, sampler)
            q_i, offset = processGroverOutput(state_i, mQubits, mBitsIntPart)
            
            count+=1
            print(count, q_i, offset)
            
            if offset<0:
                tot_offset += offset
                Found = True
                q_min = q_i
                count = 0
                break
            
            T = int(np.ceil(5/4*T)) #see https://www.youtube.com/watch?v=hnpjC8WQVrQ (50')

    return q_min, tot_offset

### Our algorithm

In [8]:
def OptimizeFunc(A, b, c, d, mBitsIntPart, mQubits, p, iteGroverMax, passManager, sampler):
    """
    Optimizes a quadratic function using Grover's Algorithm and returns the arguments and corresponding minimim function value.

    Parameters:
        A (np.ndarray): Matrix constant of the quadratic function x.T @ A @ x + b.T @ x + c.
        b (np.ndarray): Vector constant of the quadratic function x.T @ A @ x + b.T @ x + c.
        c (float): Scalar constant of the quadratic function.
        d (int): Dimension of the problem.
        mBitsIntPart (int): Number of bits of the integer part of the function values.
        mQubits (int): Number of qubits used for the integer function value.
        p (int): Number of cells in the grid for each dimension. p+1 must be a power of 2.
        iteGroverMax (int): Maximum number of Grover subroutine iterations.
        passManager (PassManager): Qiskit pass manager for circuit optimization.
        sampler (Sampler): Qiskit sampler for running the circuit.

    Returns:
        tuple: A tuple (xOpt, FvalOpt) where xOpt is the value of the maximal argument and FvalOpt is the maximal function value.
    """ 
    
    nQubits = int(np.log2(p+1))

    # Transforming to binary problem:
    newA, newb = createCoefficientsF(A, b, p, nQubits, removeDiag = True)

    # Random choice of an initial max state:
    qarg = np.random.randint(2, size=d*nQubits)
    Fval = qarg.T @ newA @ qarg + newb.T@qarg + c

    initialState = (qarg, Fval)
    nElements = 2**(d*nQubits)
    
    # GAS
    StatePhaseSuperpos = statesPhasePreparator(d, nQubits, mBitsIntPart, mQubits, newA, newb, c)
    q_min, Fq_min = GroverAdaptiveSearch(StatePhaseSuperpos, d, nQubits, mQubits, mBitsIntPart, 2**(d*nQubits),
                                               initialState, iteGroverMax, passManager, sampler)
    x_min = np.zeros(d)
    for i in range(d):
        x_min[i] = binary2int(q_min[i*nQubits:(i+1)*nQubits])/p
    
    return x_min, Fq_min

## Test of our algorithm

In [9]:
backend = Aer.get_backend('qasm_simulator')
target = backend.target

pm = generate_preset_pass_manager(target=target, optimization_level=3)
sampler = Sampler()

In [ ]:
p = 7
mQubits = 8+1
mBitsIntPart = 2
d=2

A = np.matrix([[0,-2],[0,0.5]])
b = np.array([0,-0.5])
c = 1.5
iteGroverMax = 10


x, F = OptimizeFunc(A, b, c, d, mBitsIntPart, mQubits, p, iteGroverMax, pm, sampler)

1 111101 -1.09375
1 100101 0.609375
2 110000 1.515625
3 011110 0.71875
4 110111 -0.203125
1 011101 1.015625
2 001111 1.421875
3 111111 -0.28125
1 010101 1.5
2 110000 2.0
3 101001 1.734375


In [ ]:
x, F